# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
read_path = os.path.join("output_data/cities.csv")
main_frame = pd.read_csv(read_path, low_memory=False)
main_frame.head()

,City_ID,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
0,0,IN,1584214774,19.82,84.80,99,76,73.89,6.49
1,1,RU,1584214125,70.63,147.92,69,98,-28.14,4.25
2,2,AU,1584214737,-33.87,121.90,75,88,64.40,16.11
3,3,CK,1584214095,-21.21,-159.78,82,78,82.40,5.82
4,4,AU,1584214189,-33.65,115.33,83,98,66.00,13.27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
type(main_frame['Humidity'][1])

numpy.int64

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# store locations
locations = main_frame[['Latitude', 'Longitude']]

# Fill NaN values and convert to float
humidity = main_frame["Humidity"].astype(float)

In [5]:
main_frame["Max Temp"].max()

104.0

In [6]:
# plot gmaps
figure = gmaps.figure() # this is to make a base gmap

# add humidity layer on top of gmap
humidity_layer = gmaps.heatmap_layer(locations,
                                     weights = humidity,  # == weights
                                     dissipating = True,
                                     max_intensity = 105,
                                     point_radius = 6
                                     )
figure.add_layer(humidity_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [58]:
# filter weather conditions:
maxtemp_df = main_frame.loc[(main_frame['Max Temp'] < 80) &
                             (main_frame['Max Temp'] > 70) &
                             (main_frame['Cloudiness'] == 0) &
                             (main_frame['Wind Speed'] < 10),
                              :]
maxtemp_df.dropna(how='any', inplace=True)


C:\Users\diamo\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [51]:
# quick dummy check to make sure the cloudiness in the entire main_frame is not 100% zero before being filtered into maxtemp_df
main_frame['Cloudiness'].describe()

count    576.000000
mean      54.163194
std       38.662298
min        0.000000
25%       20.000000
50%       63.500000
75%       92.000000
max      100.000000
Name: Cloudiness, dtype: float64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
# store into hotel_df (= maxtemp_df)
hotel_df = maxtemp_df
hotel_df

,City_ID,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
52,52,MR,1584214543,20.50,-10.07,0,7,71.69,4.61
173,173,NE,1584214651,18.74,7.39,0,11,75.11,8.34
193,193,BR,1584214305,-24.79,-50.01,0,64,72.05,1.92
255,255,IN,1584214813,16.78,80.30,0,86,72.41,4.27
298,298,CM,1584214819,12.08,15.03,0,11,78.80,5.82
307,307,YE,1584214821,13.17,44.58,0,69,73.17,7.56
425,425,MZ,1584214839,-24.53,32.98,0,76,77.13,7.45


In [65]:
bk = []
for index, row in hotel_df.iterrows(): # 30.118634, -95.439098
    targ_lat = row['Latitude']
    targ_long =  row['Longitude']
    
    target_coordinates = f'{targ_lat}, {targ_long}'
    

    target_radius = 32187 # search within 20 mile radius
    target_type = "lodging"

#   set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent=4, sort_keys=True))
    print(response["results"][])



SyntaxError: invalid syntax (<ipython-input-65-63856b0d545e>, line 25)

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display Map